In [39]:
from music21 import *

In [4]:
def major_scale(tonic: pitch.Pitch, pathological=True, isMinor = False) -> list[pitch.Pitch]:
    '''
    Given a tonic pitch.Pitch(), return a list of music21 pitch.Pitch 
    objects that form one octave of a major scale above that pitch,
    including the tonic pitch at the beginning and the tonic pitch up an octave at the end.

    (this is an example of a complete or nearly-complete docstring
    that does not need to be filled in.)
    '''
    
    '''
    Note that there might be pathological names for keys, like E# major and 
    B-- major as indicated. The problem doesn't specify how we should
    treat these: should we actually write something like "E# F## ..."
    or should we first convert to E# = F and then the answer is "F G ..."?
    
    I choose the latter. This is for the following reasons:
    
    1) Nobody would actually say E# major or B-- major in practice.
    The closest we get to something 'uncomfortable' is A# major, 
    but that's enharmonic in B-flat major, and besides, A# major
    is just uncommon but not simplifiable.
    
    2) How sholud we represent "E#"'s other notes if we stick to it?
    Should we go "E# F## G## A# B## ..."
    or should it be
    "E# G A B- C ..."?
    
    We could follow the standard rule by increasing by 2 or 1 semitones
    as necessary, and making sure the 'base' letter (e.g. base letter of 
    "A---" is A) follows the EFGABCDE pattern, but...why?
    
    I mean, it is at least consistent. I don't think "E# G A B- ..."
    will ever be seen.
    
    As a result, I have allowed my program to take BOTH results
    based on the preference of the user.
    
    We can have the normal thing: first, change E# to F, and then keep going.
    Otherwise, we can have the pathological thing: E# F## G## ...
    
    No matter what, the EFGABCDE pattern is the same.
    '''
    
    #first, a few helpers
    def letter(name: str) -> str:
        '''
        Gets the letter name of the pitch.
        '''
        return name[0]
    
    def modif(name: str) -> str:
        '''
        Gets the accidentals of the pitch.
        '''
        result = ""
        for i in range(1, len(name)):
            if name[i] not in '0123456789':
                result += name[i]
            else:
                return result
        return result
    
    def octave(name: str) -> str:
        '''
        returns the octave of the pitch.
        '''
        # print('octave', name)
        numPos = 1
        for i in range(1, len(name)):
            if name[i] in '0123456789':
                numPos = i
                break
        return name[numPos:]
    
    simple = tonic
    if not pathological:
        simple = tonic.simplifyEnharmonic()
    
    #now, start generating the rest of the sequence.
    tones = [2, 2, 1, 2, 2, 2, 1] #how many half-steps to go up
    if isMinor:
        tones = [2, 1, 2, 2, 1, 2, 2]
    
    #how many half-steps to get up to the next letter without changing the symbol
    halfs = {'C':2, 'D':2, 'E':1, 'F':2, 'G':2, 'A':2, 'B':1}
    
    letters = 'CDEFGAB'
    
    maj_scale = [0] * 8
    maj_scale[0] = simple
    #yes, there is an issue about referencing the tonic...we don't change it so I won't worry about it
    
    cur_pitch = simple.nameWithOctave #dealing with names is easy, also pitch is an expensive operation
    for i in range(7):
        # print('Current pitch', cur_pitch)
        #repeat this process until we get to the top
        movement = tones[i] #this is how much we need to move up
        curLetter = letter(cur_pitch)
        curOctave = octave(cur_pitch)
        # print('current letter', curLetter, modif(cur_pitch), 'current octave', curOctave)
        half_needed = halfs[curLetter] #how many half-tones we need to move
        # print(curLetter, curOctave)
        
        differential = movement - half_needed #difference between these, either -1, 0, or 1
        # print('movement to get to next tone in major', movement, 'half needed', half_needed)
        
        newPitchStr = ""
        newLetter = ""
        newOctave = ""
        newAccidental = ""
        
        #no matter what our differential is, the new letter is clear. also check octave.
        
        letterPos = letters.index(curLetter)
        # print('current letter', curLetter, 'letter position', letterPos)
        if letterPos != len(letters) - 1:
            '''
            not going to be out of bounds
            '''
            newLetter = letters[letterPos + 1]
            newOctave = curOctave
        else:
            # will be out of bounds.
            newLetter = letters[0]
            newOctave = str(int(curOctave) + 1)
            
        # print('new letter', newLetter, 'newoctave', newOctave)
        
        accidental = modif(cur_pitch)
        # print('cur accidental', accidental)
        #now, the accidentals
        newAccidental = accidental
        # print('new accidental', newAccidental)
        # print('DIFFERENTIAL', differential)

        if differential == 1:
            '''
            move the accidental UP one.
            '''            
            if accidental == '':
                #nothing. add sharp
                newAccidental += '#' 
            if '#' in accidental:
                newAccidental = accidental + '#'
            if '-' in accidental:
                newAccidental = (accidental.count('-') - 1) * '-'
        
        if differential == -1:
            #negative differential, move down one            
            if accidental == '':
                newAccidental += "-"
            if '#' in accidental:
                newAccidental = (accidental.count('#') - 1) * '#'
            if '-' in accidental:
                newAccidental = accidental + '-'
        
        # print('new accidental', newAccidental)
        cur_pitch = newLetter  + newAccidental + newOctave
        maj_scale[i+1] = pitch.Pitch(cur_pitch)
    
    return maj_scale

In [10]:
def mutate_melody(score: stream.Score(), melody: stream.Part(), tonic: pitch.Pitch) -> stream.Part():
    '''
    given a melody, change it as appropriate. then return that melody.
    '''
    pass

# Helper Functions

They will help us for some basic tasks.

In [ ]:
def note_without_octave(n: note.Note) -> str:
    '''
    given a note, 
    '''

In [ ]:
def next_note_up(n: note.Note, tonic. pitch.Pitch) -> note.Note:
    '''
    given a note in a scale, find the next note that's one higher.
    '''
    

# Modifications to Melody

They are all intentionally short, and meant to be modular--that is, they can build on top of each other without causing much trouble.

## Articulation Modifications

In [61]:
def staccato_short(melody: stream.Part(), min_dur: float):
    '''
    make all sufficiently-short notes in the melody staccato.
    '''
    for n in melody[note.Note]:
        if n.quarterLength <= min_dur:
            n.articulations += [articulations.Staccato()]

In [67]:
def grace_beginners(melody: stream.Part(), tonic: pitch.Pitch, offset_mult=4):
    '''
    given a melody, add a grace note to notes which have the desired offset
    multiplier (default 4; grace at 0, 4, 8, etc.)
    
    This method is to be called last!
    '''
    sc = scale.MajorScale(tonic.name)
    
    for n in melody[note.Note]:
        if n.offset % offset_mult == 0:
            grace_start = note.Note(sc.nextPitch(n.nameWithOctave))
            grace = grace_start.getGrace()
            melody.insert(n.offset, grace)

In [63]:
def accent_downbeat(melody: stream.Part()):
    '''
    given a melody, accent every downbeat of the measures of the melody.
    '''
    for n in melody[note.Note]:
        n.articulations += [articulations.Accent()]

In [60]:
n = note.Note('G4')
n
melody = stream.Part()
melody.append(n)
accent_downbeat(melody)
grace_beginners(melody, pitch.Pitch('C4'))
for no in melody[note.Note]:
    print(no)

<music21.note.Note A>
<music21.note.Note G>


In [43]:
sc = scale.MajorScale('A')
sc.nextPitch(note.Note('F4').nameWithOctave)

<music21.pitch.Pitch F#4>

## Involvement

The melody gets thicker...or less so.

In [53]:
def double_note_frequency(melody: stream.Part(), tonic: pitch.Pitch) -> stream.Part():
    '''
    given a melody, change each note to have two notes, each with double the frequency
    '''
    
    newm = stream.Part()
    for n in melody[note.Note]:
        n1 = copy.deepcopy(n)
        n2 = copy.deepcopy(n1)
        
        n1.quarterLength = n.quarterLength * 0.5
        #this will modify both of them correctly
        
        #delete note n
        #add to that place: n1, n2 sequentially
        newm.append(n1)
        newm.append(n2)
    
    return newm

In [57]:
# test double_note_frequency
melody = stream.Part()
melody.append(note.Note('G4'))
melody.append(note.Note('G#4'))

newm = double_note_frequency(melody, pitch.Pitch('G4'))
for n in newm[note.Note]:
    print(n, n.offset)

<music21.note.Note G> 0.0
<music21.note.Note G> 0.5
<music21.note.Note G#> 1.5
<music21.note.Note G#> 2.0


In [79]:
def elevate_repeats(melody: stream.Part(), tonic: pitch.Pitch, offset_mult=4) -> stream.Part():
    '''
    given a melody, for every note which starts a measure or a given offset
    multiplier (default 4: start of a 4/4 measure), change that note
    to have the next note up for the last 1/4 duration.
    
    Inspired by Country Gardens.
    
    C(4)-C(4) becomes C(3)-D(1)-C(4); parentheses illustrate durations.
    '''
    
    sc = scale.MajorScale(tonic.name)
    
    for n in melody[note.Note]:
        if n.offset % offset_mult == 0:
            dur = n.quarterLength
            n.quarterLength = 0.75 * dur
            add_note = note.Note(sc.nextPitch(n.nameWithOctave))
            add_note.quarterLength = 0.25 * dur
            melody.insert(n.offset + 0.75*dur, add_note)

In [168]:
melody = stream.Part()
melody.append(note.Note('G4'))
melody.append(note.Note('G4'))
melody.append(note.Note('G4'))
melody.append(note.Note('G4'))
melody.append(note.Note('G4'))
print(note.Note('G4').quarterLength)
melody
elevate_repeats(melody, pitch.Pitch('C4'))
for no in melody[note.Note]:
    print(no, no.offset)

1.0
<music21.note.Note G> 0.0
<music21.note.Note A> 0.75
<music21.note.Note G> 1.0
<music21.note.Note G> 2.0
<music21.note.Note G> 3.0
<music21.note.Note G> 4.0
<music21.note.Note A> 4.75


## Stacking

Keep the same notes, but thicken them.

In [167]:
def chord_it(melody: stream.Part(), tonic: pitch.Pitch, is_above=False, offset_mult=4) -> stream.Part():
    '''
    given a melody, take the first note of each measure and turn that into a chord
    '''
    sc = scale.MajorScale(tonic.name)
    
    for n in melody[note.Note]:
        chord_notes = []
        if n.offset % offset_mult == 0:
            pitch_nums = [2, 4, 7]
            for i in range(3):
                new_note = note.Note(sc.nextPitch(n.nameWithOctave, stepSize=pitch_nums[i]).nameWithOctave)
                new_note.quarterLength = n.quarterLength
                if not is_above:
                    new_note.octave -= 1 if new_note.name != n.name else 2
                chord_notes.append(new_note)
            new_chord = chord.Chord(chord_notes)
            melody.insert(n.offset, new_chord)

In [169]:
chord_it(melody, pitch.Pitch('C4'))

for no in melody[chord.Chord]:
    print(no, no.pitches, no.offset)

melody.show('midi')

<music21.chord.Chord B3 D4 G3> (<music21.pitch.Pitch B3>, <music21.pitch.Pitch D4>, <music21.pitch.Pitch G3>) 0.0
<music21.chord.Chord B3 D4 G3> (<music21.pitch.Pitch B3>, <music21.pitch.Pitch D4>, <music21.pitch.Pitch G3>) 4.0


In [149]:
n = note.Note('G6')

p = pitch.Pitch('G6')

p.octave

6

# Scaling the Hierarchy

Becuse parts cannot simply be concatenated to each other, we will need a few helper functions.

In [21]:
import copy

melody = stream.Part()
melody.append(note.Note('G4'))
melody.append(note.Note('F4'))

melody2 = stream.Part()
melody2.append(note.Note('G43'))
melody2.append(note.Note('F43'))

melody3 = melody + melody2

for n in melody3[note.Note]:
    print(n, n.octave, n.offset)

<music21.note.Note G> 4 0.0
<music21.note.Note G> 43 0.0
<music21.note.Note F> 4 1.0
<music21.note.Note F> 43 1.0


# Testing

We will test our function on pieces of different genres and see how they compare.

In [9]:
# primary test: Bach!

bach = corpus.parse('bwv66.6')

bach_melody = bach[1]

tonic = pitch.Pitch('A4')

changed_melody = mutate_melody(bach, bach_melody, tonic)

<music21.stream.Part Soprano>